In [1]:
import pandas as pd
import h3
import shapely
import geopandas as gpd
import numpy as np

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
# Read the CSV file into a Pandas DataFrame
file_path = 'loit.csv'  
df = pd.read_csv(file_path, encoding = 'utf-8')

# Define a list of column names to select
selected_column_names = ['start', 'end','name', 'region', 'flag', 'lat', 'lon']

# Select the specified columns from the DataFrame
df = df[selected_column_names]

# Define a dictionary to map the old column names to the new names
column_name_mapping = {
    'name': 'Name',
    'region': 'Regions',
    'flag': 'Country',
    'lat': 'Latitude',
    'lon': 'Longitude',
    'start':'Start',
    'end': 'End'
}

# Use the rename method to rename the columns
df.rename(columns=column_name_mapping, inplace=True)
df['Start']=pd.to_datetime(df['Start'], format='ISO8601')
df['End']=pd.to_datetime(df['End'], format='ISO8601')
df['hour'] = df.apply(lambda row: pd.date_range(row['Start'], row['End'], freq='H'), axis=1)
df = df.explode('hour').reset_index() 
# Display the first few rows of the DataFrame
df.head()

,index,Start,End,Name,Regions,Country,Latitude,Longitude,hour
0,0,2022-10-06 07:58:16.760000+00:00,2024-03-25 23:57:42+00:00,'LS361',"{'mpa': [], 'eez': ['8486'], 'rfmo': ['APFIC',...",'CHN',27.1495,121.9008,2022-10-06 07:58:16.760000+00:00
1,0,2022-10-06 07:58:16.760000+00:00,2024-03-25 23:57:42+00:00,'LS361',"{'mpa': [], 'eez': ['8486'], 'rfmo': ['APFIC',...",'CHN',27.1495,121.9008,2022-10-06 08:58:16.760000+00:00
2,0,2022-10-06 07:58:16.760000+00:00,2024-03-25 23:57:42+00:00,'LS361',"{'mpa': [], 'eez': ['8486'], 'rfmo': ['APFIC',...",'CHN',27.1495,121.9008,2022-10-06 09:58:16.760000+00:00
3,0,2022-10-06 07:58:16.760000+00:00,2024-03-25 23:57:42+00:00,'LS361',"{'mpa': [], 'eez': ['8486'], 'rfmo': ['APFIC',...",'CHN',27.1495,121.9008,2022-10-06 10:58:16.760000+00:00
4,0,2022-10-06 07:58:16.760000+00:00,2024-03-25 23:57:42+00:00,'LS361',"{'mpa': [], 'eez': ['8486'], 'rfmo': ['APFIC',...",'CHN',27.1495,121.9008,2022-10-06 11:58:16.760000+00:00


In [4]:
def get_hexagon_grid(latitude, longitude, resolution):
    """
    Generate a hexagonal grid GeoDataFrame centered around a specified location.
    Parameters:
    - latitude (float): Latitude of the center point.
    - longitude (float): Longitude of the center point.
    - resolution (int): H3 resolution for hexagons.
    - ring_size (int): Number of rings to create around the center hexagon.
    Returns:
    - hexagon_df (geopandas.GeoDataFrame): GeoDataFrame containing hexagons and their geometries.
    """

    # Get the H3 hexagons covering the specified location
    hexagons= []
    for i, row in df.iterrows():
        hexagons.append(h3.geo_to_h3(row.Latitude, row.Longitude, resolution))  # Convert the set to a list

    # Create a GeoDataFrame with hexagons and their corresponding geometries
    hexagon_geometries = [shapely.geometry.Polygon(h3.h3_to_geo_boundary(hexagon, geo_json=True)) for hexagon in hexagons]
    hexagon_df = gpd.GeoDataFrame({'Hexagon_ID': hexagons, 'geometry': hexagon_geometries})
    df['Hexagon_ID']=hexagons
    return hexagon_df

In [5]:
df

,index,Start,End,Name,Regions,Country,Latitude,Longitude,hour
0,0,2022-10-06 07:58:16.760000+00:00,2024-03-25 23:57:42+00:00,'LS361',"{'mpa': [], 'eez': ['8486'], 'rfmo': ['APFIC',...",'CHN',27.1495,121.9008,2022-10-06 07:58:16.760000+00:00
1,0,2022-10-06 07:58:16.760000+00:00,2024-03-25 23:57:42+00:00,'LS361',"{'mpa': [], 'eez': ['8486'], 'rfmo': ['APFIC',...",'CHN',27.1495,121.9008,2022-10-06 08:58:16.760000+00:00
2,0,2022-10-06 07:58:16.760000+00:00,2024-03-25 23:57:42+00:00,'LS361',"{'mpa': [], 'eez': ['8486'], 'rfmo': ['APFIC',...",'CHN',27.1495,121.9008,2022-10-06 09:58:16.760000+00:00
3,0,2022-10-06 07:58:16.760000+00:00,2024-03-25 23:57:42+00:00,'LS361',"{'mpa': [], 'eez': ['8486'], 'rfmo': ['APFIC',...",'CHN',27.1495,121.9008,2022-10-06 10:58:16.760000+00:00
4,0,2022-10-06 07:58:16.760000+00:00,2024-03-25 23:57:42+00:00,'LS361',"{'mpa': [], 'eez': ['8486'], 'rfmo': ['APFIC',...",'CHN',27.1495,121.9008,2022-10-06 11:58:16.760000+00:00
...,...,...,...,...,...,...,...,...,...
141579,9999,2024-01-12 19:35:00+00:00,2024-01-13 20:09:53+00:00,'ZHELINGYU69361',"{'mpa': [], 'eez': ['8486'], 'rfmo': ['IWC', '...",'CHN',27.8944,121.9663,2024-01-13 15:35:00+00:00
141580,9999,2024-01-12 19:35:00+00:00,2024-01-13 20:09:53+00:00,'ZHELINGYU69361',"{'mpa': [], 'eez': ['8486'], 'rfmo': ['IWC', '...",'CHN',27.8944,121.9663,2024-01-13 16:35:00+00:00
141581,9999,2024-01-12 19:35:00+00:00,2024-01-13 20:09:53+00:00,'ZHELINGYU69361',"{'mpa': [], 'eez': ['8486'], 'rfmo': ['IWC', '...",'CHN',27.8944,121.9663,2024-01-13 17:35:00+00:00
141582,9999,2024-01-12 19:35:00+00:00,2024-01-13 20:09:53+00:00,'ZHELINGYU69361',"{'mpa': [], 'eez': ['8486'], 'rfmo': ['IWC', '...",'CHN',27.8944,121.9663,2024-01-13 18:35:00+00:00


In [6]:
lat = 24.7
lng = 120.3

# Generate H3 hexagons at a specified resolution (e.g., 9)
resolution = 5



# Hexagon shapes for vessels
hexagon_df = get_hexagon_grid(lat, lng, resolution)

# Visualize the first rows of the GeoDataFrame
hexagon_df.head()

,Hexagon_ID,geometry
0,854b144bfffffff,"POLYGON ((121.80966 27.13923, 121.82689 27.062..."
1,854b144bfffffff,"POLYGON ((121.80966 27.13923, 121.82689 27.062..."
2,854b144bfffffff,"POLYGON ((121.80966 27.13923, 121.82689 27.062..."
3,854b144bfffffff,"POLYGON ((121.80966 27.13923, 121.82689 27.062..."
4,854b144bfffffff,"POLYGON ((121.80966 27.13923, 121.82689 27.062..."


In [29]:
grouped_df = df.groupby('hour').count('Hexagon_ID')
#grouped_df = grouped_df.groupby.agg({'Name': ['count', '<br>'.join]}).reset_index()
# Rename columns for clarity
#grouped_df.columns = ['Hexagon_ID', 'hour', 'Count', 'Vessels']
#grouped_df.replace("''", np.nan,regex=True, inplace=True)
# Visualize the first rows of the DataFrame
#grouped_df=grouped_df.dropna()
grouped_df.head(50)


TypeError: count() takes 1 positional argument but 2 were given

In [8]:

import plotly.express as px
def create_choropleth_map(geojson_df, data_df, alpha=0.4, map_style="carto-positron", color_scale="Viridis"):
    """
    Create an interactive choropleth map using Plotly Express.
    Parameters:
    - geojson_df (GeoDataFrame): GeoJSON data containing polygon geometries.
    - data_df (DataFrame): DataFrame containing data to be visualized on the map.
    - alpha (float): Opacity level for the map polygons (0.0 to 1.0).
    - map_style (str): Map style for the Plotly map (e.g., "carto-positron").
    - color_scale (str): Color scale for the choropleth map.
    Returns:
    None
    """
    # Merge the GeoJSON data with your DataFrame
    merged_df = geojson_df.merge(data_df, on="Hexagon_ID", how="left")

    # Create a choropleth map using px.choropleth_mapbox
    fig = px.choropleth_mapbox(
        merged_df,
        geojson=merged_df.geometry,
        locations=merged_df.index,  # Use index as locations to avoid duplicate rows
        color="Count",
        color_continuous_scale=color_scale,
        title="Vessel Distribution Around Taiwan",
        mapbox_style=map_style,
        animation_frame='hour',
        center={"lat": lat, "lon": lng},  # Adjust the center as needed
        zoom=5,
    )

    # Customize the opacity of the hexagons
    fig.update_traces(marker=dict(opacity=alpha))

    # Add hover data for hotel names
    fig.update_traces(customdata=merged_df["Vessels"])

    # Define the hover template 
    hover_template = "<b>Vessels:</b> %{customdata}<extra></extra>"
    fig.update_traces(hovertemplate=hover_template)

    # Set margins to 25 on all sides
    fig.update_layout(margin=dict(l=35, r=35, t=45, b=35))
    
    # Adjust the width of the visualization
    fig.update_layout(width=1000) 

    fig.show()

# Call the function with your GeoJSON and DataFrame
create_choropleth_map(geojson_df=hexagon_df, data_df=grouped_df)

KeyboardInterrupt: 